In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
def plot_matrix(x, y, group, fmt='.', **kwargs):
    """
    Given two d-dimensional datasets of n points,
    makes a figure containing d x d plots, where the (i, j) plot
    plots the ith dimension against the jth dimension.
    """

    x = np.asarray(x)
    y = np.asarray(y)
    group = np.squeeze(np.asarray(group))
    n, p = x.shape
    n_, q = y.shape
    n__, = group.shape
    assert n == n_ == n__
    groups = sorted(set(group))
    if isinstance(fmt, str):
        fmt = {k: fmt for k in groups}
    fig, axes = plt.subplots(p, q, squeeze=False, **kwargs)
    for i, axrow in enumerate(axes):
        for j, ax in enumerate(axrow):
            for g in groups:
                ax.plot(x[group == g, i], y[group == g, j], fmt[g])
            if len(axes) > 2:
                ax.locator_params(tight=True, nbins=4)

def plot_groups(x, group, fmt='.', **kwargs):
    """
    Helper function for plotting a 2-dimensional dataset with groups
    using plot_matrix.
    """
    n, d = x.shape
    assert d == 2
    x1 = x[:, 0].reshape(n, 1)
    x2 = x[:, 1].reshape(n, 1)
    plot_matrix(x1, x2, group, fmt, **kwargs)

In [3]:
import sklearn.datasets
iris = sklearn.datasets.load_iris()
data = iris['data']
labels = iris['target']


In [14]:
from scipy.stats import multivariate_normal
def pdf(points, mean, cov, prior):
    points, mean, cov = np.asarray(points), np.asarray(mean), np.asarray(cov)
    prior = np.asarray(prior)
    n, d = points.shape
    k, d_1 = mean.shape
    k_2, d_2, d_3 = cov.shape
    k_3, = prior.shape
    assert d == d_1 == d_2 == d_3
    assert k == k_2 == k_3, "%s %s %s should be equal" % (k, k_2, k_3)

    # Compute probabilities
    prob = []
    for i in range(k):
        if prior[i] < 1 / k ** 3:
            prob.append(np.zeros(n))
        else:
#            print("pdf mean ",mean[i])
#            print("pdf cov ",cov[i])
            prob.append(
                prior[i] *
                multivariate_normal.pdf(
                    mean=mean[i], cov=cov[i], x=points))
    prob = np.transpose(prob)  # n x k
    # Normalize cluster probabilities of each point
    prob = prob / np.sum(prob, axis=1, keepdims=True)  # n x k

    assert prob.shape == (n, k)
    assert np.allclose(prob.sum(axis=1), 1)
    return prob

In [5]:
def most_likely(points, mean, cov, prior):
    prob = pdf(points, mean, cov, prior)
    return np.argmax(prob, axis=1)

In [17]:
def em(points, k, epsilon, mean=None):
    points = np.asarray(points)
    n, d = points.shape

    # Initialize and validate mean
    if mean is None:
        # Randomly pick k points
        maximum = np.amax(points, axis = 0)
        minimum = np.amin(points, axis = 0)
        mean = np.zeros((k,d))
        for i in range(k):
            for j in range(d):
                mean[i,j] = np.random.uniform(minimum[j], maximum[j])

    # Validate input
    mean = np.asarray(mean)
    k_, d_ = mean.shape
    assert k == k_
    assert d == d_

    # Initialize cov, prior
    cov = np.array([np.eye(d) for x in range(k)])
    prior = np.array([1/k for x in range(k)])

    tired = False
    old_mean = np.zeros_like(mean)
    while not tired:
        old_mean[:] = mean

        # Expectation step
 #       print("--------------------------------")
 #       print("Emean ",mean)
 #       print("Ecov ",cov)
 #       print("Eprior ",prior)
        weights=pdf(points, mean, cov, prior)
 #       print ("weights ",weights)
        # Maximization step
        weights_sum=np.sum(weights,axis=0)
 #       print("sum",weights_sum)
        for i in range(k):
            res=weights[:,i].reshape(n,1)*points
            mean[i]=np.sum(res,axis=0)/weights_sum[i]
            new_cov=np.zeros([d,d])
            for j in range(n):
                new_cov+=weights[j,i]*np.outer(points[j]-mean[i],points[j]-mean[i])
            cov[i]=new_cov/weights_sum[i]
        prior[:]=weights_sum/n    
                    

        # Finish condition
        dist = np.sqrt(((mean - old_mean) ** 2).sum(axis=1))
        tired = np.all(dist < epsilon)

    # Validate output
    assert mean.shape == (k, d)
    assert cov.shape == (k, d, d)
    assert prior.shape == (k,)
    return mean, cov, prior

In [19]:
import sklearn.datasets
import sklearn.decomposition
tmean=np.array([[-3.59,0.25],[-1.09,-0.46],[0.75,1.07]])
pca = sklearn.decomposition.PCA(2)
data_pca = pca.fit_transform(data)
rmean,rcov,rprior=em(data_pca,3,0.001,mean=tmean)
print(rmean)
print(rcov)
print(rprior)

[[-2.64084012  0.19052331]
 [ 0.48520361 -0.12609158]
 [ 1.44765296 -0.09056442]]
[[[ 0.04777048  0.05590727]
  [ 0.05590727  0.21472044]]

 [[ 0.18269699  0.19662482]
  [ 0.19662482  0.21385676]]

 [[ 1.04886054  0.36815436]
  [ 0.36815436  0.22809615]]]
[ 0.33333233  0.08813558  0.57853209]


In [4]:
a=np.array([[1,5],[2,1],[5,4],[8,2]])
em(a,2,0.5)

--------------------------------
Emean  [[ 4.56820839  2.53595853]
 [ 2.08299319  2.78803037]]
Ecov  [[[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]]
Eprior  [ 0.5  0.5]
pdf mean  [ 4.56820839  2.53595853]
pdf cov  [[ 1.  0.]
 [ 0.  1.]]
pdf mean  [ 2.08299319  2.78803037]
pdf cov  [[ 1.  0.]
 [ 0.  1.]]
weights  [[  1.71100173e-03   9.98288998e-01]
 [  5.33800177e-02   9.46619982e-01]
 [  9.78625886e-01   2.13741141e-02]
 [  9.99992373e-01   7.62738277e-06]]
sum [ 2.03370928  1.96629072]
--------------------------------
Emean  [[ 6.39301772  2.93868125]
 [ 1.52493246  3.0634212 ]]
Ecov  [[[ 2.73456884 -1.23895261]
  [-1.23895261  1.07750682]]

 [[ 0.37998455 -0.95268636]
  [-0.95268636  3.9633553 ]]]
Eprior  [ 0.50842732  0.49157268]
pdf mean  [ 6.39301772  2.93868125]
pdf cov  [[ 2.73456884 -1.23895261]
 [-1.23895261  1.07750682]]
pdf mean  [ 1.52493246  3.0634212 ]
pdf cov  [[ 0.37998455 -0.95268636]
 [-0.95268636  3.9633553 ]]
weights  [[  4.63292361e-03   9.95367076e-01]
 [  1.

(array([[ 6.48728888,  3.00462221],
        [ 1.50116092,  2.99535633]]), array([[[ 2.31454955, -1.52189672],
         [-1.52189672,  1.00691198]],
 
        [[ 0.24999865, -0.99999461],
         [-0.99999461,  3.99997844]]]), array([ 0.50115823,  0.49884177]))